<a href="https://colab.research.google.com/github/enmwmak/Teaching/blob/main/EIE4435/Programs/Speech_Enhance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This script implements a simple spectral subtraction algorithm. After setting the variables, you may click "Runtime > Run all" (if you use Colab) or click the "Run All" button (if you use Visual Studio Code) to run all cells in one operation.

## **Procedures**
1. Read the codes in this .ipynb file to understand how they work.
1. Run all cells. Listen to the audio of (1) the noisy speech and (2) the denoised speech.
1. Modify the variable '*alpha*' to see its effect on the denoised speech.
1. Try other speech files and repeat Steps 2 and 3.

## **Answer the following questions**
1. Identify the regions in the spectrogram of the denoised speech that correspond to the musical noise.
1. Explain why spectral subtraction causes musical noise in the denoised speech.
1. Identify the artifact and explain why such artifact exists when '*alpha*' is very large, say *alpha*=3'.
1. Explain why adding a noise floor to the denoised signal can help alleviate the artifact in Step 3.
1. Will adding a small amount of background noise to the denoised signal help alleviating the musical noise? Try it by modifying the program.
1. Explain why spectral subraction is not good at removing the machine-gun noise.
1. Explain why spectral subtraction will not cause musical noise in the enhanced speech if the SNR is very high.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import librosa
from IPython.display import Audio

In [ ]:
# Define some variables and constants
use_colab = True            # Set to false if you use IDE such as Visual Studio Code
infile = '../AudioFiles/noisy_speech2.wav'
alpha = 2                   # Over-subtraction factor
frame_size = 512
frame_shift = 64
noise_dur = 0.5             # Duration at the beginning of file considered as noise

In [ ]:
# Mount Google Drive and create a folder for the lab exercise
if use_colab == True:
  from google.colab import drive
  drive.mount('/content/drive')
  !mkdir -p /content/drive/MyDrive/Learning/EIE4435/Programs
  %cd /content/drive/MyDrive/Learning/EIE4435/Programs

In [ ]:
# Function for plotting speech signal and its spectrogram
def plot_speech(x, sr=8000, frm_len=512, hop_len=256):
    X = librosa.amplitude_to_db(np.abs(librosa.stft(x, n_fft=frm_len, hop_length=hop_len)), ref=np.max)  # STFT of x
    _, ax = plt.subplots(nrows=2, sharex=True, figsize=(8,4))
    librosa.display.waveshow(x, sr=sr, ax=ax[0])
    librosa.display.specshow(X, sr=sr, n_fft=frm_len, hop_length=hop_len, x_axis='time', y_axis='linear', ax=ax[1])

In [ ]:
def specsub(y, frame_size, frame_shift, sr=8000):
    Y = librosa.stft(y, n_fft=frame_size, hop_length=frame_shift)     # Short-time Fourier transform
    Ymag= np.abs(Y)         # Get magnitude (spectrogram)
    Ypha= np.angle(Y)       # Get phase

    # Assume that the beginning of the speech file contains noise only and get its average noise spectrum
    noise_mag = Ymag[:, 0:int(noise_dur*sr/frame_shift)]
    mean_noise_mag = np.mean(noise_mag, axis=1)
    Xmag = Ymag - alpha * mean_noise_mag.reshape((mean_noise_mag.shape[0],1))

    # Implement |Y(w) - B(w)| so that all negative values are set to 0
    mask = (Xmag > 0).astype(int)
    Xmag = Xmag * mask

    # Convert to complex number using the phase information of noisy speech. Then, convert to time domain using ISTFT
    Y = Xmag * np.exp(1.0j* Ypha)
    y = librosa.istft(Y, n_fft=frame_size, hop_length=frame_shift)
    return y, Y, mask

In [ ]:
# Load the noisy speech and perform spectral subtraction
y, sr = librosa.load(infile, sr=None, mono=True) # keep native sr (sampling rate) and trans into mono
x, X, mask = specsub(y, frame_size=frame_size, frame_shift=frame_shift, sr=sr)
print(X.shape)

In [ ]:
# Original noisy speech
plot_speech(y, sr=sr, frm_len=frame_size, hop_len=frame_shift)

In [ ]:
# Denoised speech
plot_speech(x, sr=sr, frm_len=frame_size, hop_len=frame_shift)

In [ ]:
# Plot the mask to show the frequencies and frames at which the spectral components (in white)
# become negative after subtraction.
plt.figure(figsize=(8,2))
librosa.display.specshow(1-mask, sr=sr, n_fft=frame_size, hop_length=frame_shift, x_axis='time', y_axis='linear')
plt.colorbar()

In [ ]:
# Listen to the noisy speech
Audio(data=y, rate=sr)

In [ ]:
# Listen to the denoised speech
Audio(data=x, rate=sr)

Add non-stationary noise to clean a speech file to create a noisy speech file. Then, apply spectral subtraction to denoise the noisy speech.

In [ ]:
# Load clean speech and noise files
from itertools import cycle, islice
clean_speech_file = '../AudioFiles/clean_speech.wav'
noise_file = '../AudioFiles/machine-gun.mp3'
clean, sr = librosa.load(clean_speech_file, sr=None, mono=True) # keep native sr (sampling rate) and trans into mono
noise, fs = librosa.load(noise_file, sr=None, mono=True)
noise = librosa.resample(noise, orig_sr=fs, target_sr=sr)
noise = np.array(list(islice(cycle(noise), clean.shape[0])))
y = clean + 0.2*noise

In [ ]:
# Perform spectral subtraction
x, X, mask = specsub(y, frame_size=frame_size, frame_shift=frame_shift, sr=sr)

In [ ]:
plot_speech(y, frm_len=frame_size, hop_len=frame_shift)

In [ ]:
# Denoised speech
plot_speech(x, frm_len=frame_size, hop_len=frame_shift)

In [ ]:
# Listen to the noisy speech
Audio(data=y, rate=sr)

In [ ]:
# Listen to the denoised speech
Audio(data=x, rate=sr)

In [ ]:
# Plot the mask to show the frequencies and frames at which the spectral components (in white)
# become negative after subtraction.
plt.figure(figsize=(8,2))
librosa.display.specshow(1-mask, sr=sr, n_fft=frame_size, hop_length=frame_shift, x_axis='time', y_axis='linear')
plt.colorbar()

## **DNN-based Speech Enhancement**
**<font color="red">This part is optional. Conduct this part if you want to learn more. You do not need to put the results of this part in your report</font>.**

This part enables you to learn how to use the pre-trained speech models from SpeechBrain to reduce the noise in speech files. You can compare the speech quality of the denoised (enhanced) speech subjectively (through listening tests) and objectively (through Short-Time Objective Intelligibility (STOI)).

Before starting this lab, you should learn the <a href="https://speechbrain.readthedocs.io/en/v1.0.3/tutorials/basics.html">basics</a> of SpeechBrain and read the <a href="https://speechbrain.readthedocs.io/en/v1.0.3/tutorials/tasks/speech-enhancement-from-scratch.html">tutorial</a> on speech enhancement. After knowing what SpeechBrain is about and how it performs speech enhancement, you need to read the procedure for using its <a href="https://speechbrain.readthedocs.io/en/v1.0.3/API/speechbrain.inference.enhancement.html">pre-trained models</a> for speech enhancement. You may also want to read the paper on <a href="https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=5713237">STOI</a>.



In [ ]:
# Check Python version (This script works on Python 3.12, Torch 2.8, and Torchaudio 2.8)
!python --version
import torch
import torchaudio
print(torch.__version__)
print(torchaudio.__version__)

In [ ]:
# Installing SpeechBrain and STOI via pip.
# You should perform this step after the expiration of each Colab session.
BRANCH = 'develop'
!python -m pip install git+https://github.com/speechbrain/speechbrain.git@$BRANCH
!pip install pystoi

In [ ]:
# Clone SpeechBrain repository to your folder.
# Skip this step if you have done this before as SpeechBrain has been installed on your folder
%cd /content/drive/MyDrive/Learning/EIE4435
!git clone https://github.com/speechbrain/speechbrain/

In [ ]:
# Define a function that plots the waveforms and spectrograms of noisy and enhanced speech
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display

def show_wav_and_spec(wavfile):
  waveform, srate = librosa.load(wavfile, sr=librosa.get_samplerate(wavfile))
  n_samples = len(waveform)
  frm_size = int(0.032 * srate)       # 32ms per frame
  frm_shift = int(0.01 * srate)       # 100Hz frame rate
  n_fft = n_samples if (n_samples < frm_size) else frm_size

  # Compute magnitude spectrogram
  magspec = abs(librosa.stft(y=waveform, n_fft=frm_size, hop_length=frm_shift))

  # Plot spectrogram and waveform
  plt.figure(figsize=(10, 4))
  plt.subplot(211)
  librosa.display.specshow(librosa.amplitude_to_db(magspec), sr=srate, y_axis='linear',
                           hop_length=frm_shift)
  plt.subplot(313)
  librosa.display.waveshow(waveform, sr=srate, offset=0)
  plt.margins(x=0)
  plt.show()

  display(Audio(wavfile, autoplay=True))

In [ ]:
# Download and use a spectral-based pre-trained SE model to enhance speech
%cd /content/drive/MyDrive/Learning/EIE4435/speechbrain/templates/enhancement

from speechbrain.inference.enhancement import WaveformEnhancement
# Model is downloaded from the speechbrain HuggingFace repo
enhancer = WaveformEnhancement.from_hparams(source="speechbrain/mtl-mimic-voicebank",
                                            savedir="tmpdir2",)
enhanced = enhancer.enhance_file("../../../AudioFiles/noisy_speech2.wav", output_filename="../../../AudioFiles/denoised_speech2.wav")

In [ ]:
# Display the spectrogram and waveform of the noisy speech file and play the audio file
show_wav_and_spec("../../../AudioFiles/noisy_speech2.wav")

In [ ]:
# Display the spectrogram and waveform of the enhanced speech file and play the audio file
show_wav_and_spec("../../../AudioFiles/denoised_speech2.wav")

##